In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import pylab as pl
from IPython import display

In [4]:
def check_dead_end(obs_map, pos, direction):
    x, y = pos
    tunnel=[]
    deadend=0
    limit = obs_map.shape[0]-1
    if direction=='down':
        for i in obs_map[x:,y]:
            tunnel.append(i)
            if i == 1:
                break
    elif direction=='up':
        for i in list(obs_map[:x+1,y])[::-1]:
            tunnel.append(i)
            if i == 1:
                break
    elif direction=='left':
        for i in list(obs_map[x,:y+1])[::-1]:
            tunnel.append(i)
            if i == 1:
                break
    elif direction=='right':
        for i in list(obs_map[x,y:]):
            tunnel.append(i)
            if i == 1:
                break

    if sum(tunnel)==0:
        tunnel.append(1)

    if direction=="down":
        if y==0:
            sidewalk1=list(obs_map[x+1:x+len(tunnel)-1, y+1])
        elif y==limit:
            sidewalk2=list(obs_map[x+1:x+len(tunnel)-1, y-1])
        else:
            sidewalk1=list(obs_map[x+1:x+len(tunnel)-1, y+1])
            sidewalk2=list(obs_map[x+1:x+len(tunnel)-1, y-1])
        check_var=y
    elif direction=="up":
        if y==0:
            sidewalk1=list(obs_map[x-len(tunnel)+2:x, y+1])
        elif y==limit:
            sidewalk2=list(obs_map[x-len(tunnel)+2:x, y-1])
        else:
            sidewalk1=list(obs_map[x-len(tunnel)+2:x, y+1])
            sidewalk2=list(obs_map[x-len(tunnel)+2:x, y-1])
        check_var=y
    elif direction=="right":
        if x==0:
            sidewalk1=list(obs_map[x+1, y+1:y+len(tunnel)-1])
        elif x==limit:
            sidewalk2=list(obs_map[x-1, y+1:y+len(tunnel)-1])
        else:
            sidewalk1=list(obs_map[x+1, y+1:y+len(tunnel)-1])
            sidewalk2=list(obs_map[x-1, y+1:y+len(tunnel)-1])
        check_var=x
    elif direction=="left":
        if x==0:
            sidewalk1=list(obs_map[x+1, y-len(tunnel)+2:y])
        elif x==limit:
            sidewalk2=list(obs_map[x-1, y-len(tunnel)+2:y])
        else:
            sidewalk1=list(obs_map[x+1, y-len(tunnel)+2:y])
            sidewalk2=list(obs_map[x-1, y-len(tunnel)+2:y])
        check_var=x

    if len(tunnel)>2:
        wall = list(np.ones(len(tunnel)-2))
        if check_var==0:
            if sidewalk1==wall:
                deadend=1
        elif check_var==limit:
            if sidewalk2==wall:
                deadend=1
        else:
            if sidewalk2==wall and sidewalk1==wall:
                deadend=1
    else:
        deadend=1

    return deadend

In [ ]:
def findPath(grid, obstacles, curPos, T, visited, path, rows, cols):
    visited[curPos[0]][curPos[1]] = True
    path.append(curPos)

    if curPos == T:
        if len(path) == rows * cols - len(obstacles):
            return path
        else:
            visited[curPos[0]][curPos[1]] = False
            path.pop()
            return None

    for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        nx = curPos[0] + dx
        ny = curPos[1] + dy

        if 0 <= nx < rows and 0 <= ny < cols and not visited[nx][ny] and grid[nx][ny] != 1:
            newPath = findPath(grid, obstacles, (nx, ny), T, visited, path, rows, cols)
            if newPath:
                return newPath

    visited[curPos[0]][curPos[1]] = False
    path.pop()
    return None